In [ ]:
import sys
sys.path.append("..")

import random
import math
import itertools
from copy import deepcopy
from io import BytesIO
from pathlib import Path
from typing import Optional, Callable, List, Tuple, Iterable, Generator, Union, Dict

import PIL.Image
import PIL.ImageDraw
import plotly
import plotly.express as px
import plotly.graph_objects as go
plotly.io.templates.default = "plotly_dark"
import numpy as np
import pandas as pd
pd.options.plotting.backend = "plotly"

from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset, IterableDataset, RandomSampler
import torchvision.transforms as VT
import torchvision.transforms.functional as VF
from torchvision.utils import make_grid
from IPython.display import display, Audio
import torchaudio
from torchaudio.io import StreamReader

from src.datasets import *
from src.algo import GreedyLibrary
from src.util.image import *
from src.util import to_torch_device, iter_batches
from src.patchdb import PatchDB, PatchDBIndex
from src.models.encoder import *
from src.util.audio import *
from src.util.files import *
from scripts.train_audio_predict import AudioPredictor

In [ ]:
@torch.no_grad()
def play(audio, shape=(200, 1024), normalize=False):
    if normalize:
        audio_max = audio.abs().max()
        if audio_max:
            audio = audio / audio_max
    display(Audio(audio.clamp(-1, 1), rate=SAMPLERATE, normalize=False))
    display(plot_audio(audio, shape))
    
def read_stream(stream, seconds: float = 3.):
    chunks = []
    max_samples = int(seconds * SAMPLERATE)
    num_samples = 0
    for chunk, in stream:
        chunks.append(chunk.mean(-1))
        num_samples += chunks[-1].shape[-1]
        if num_samples >= max_samples:
            break
    return torch.concat(chunks)[:max_samples]

# load model

In [ ]:
SLICE_SIZE = 1024
LATENT_SIZE = 128
SAMPLERATE = 44100
NUM_INPUT_SLICES = 8

    
model = AudioPredictor(
        slice_size=SLICE_SIZE,
        latent_dims=LATENT_SIZE,
        num_input_slices=NUM_INPUT_SLICES,
        encoder=EncoderConv1d.from_torch(f"../models/encoder1d/conv-1x{SLICE_SIZE}-{LATENT_SIZE}.pt"),
        decoder=EncoderConv1d.from_torch(f"../models/encoder1d/conv-1x{SLICE_SIZE}-{LATENT_SIZE}-decoder.pt"),
        #predictor=nn.Linear(LATENT_SIZE * NUM_INPUT_SLICES, LATENT_SIZE),
        predictor=nn.Sequential(
            nn.Linear(LATENT_SIZE * NUM_INPUT_SLICES, LATENT_SIZE * NUM_INPUT_SLICES),
            nn.ReLU(),
            nn.Linear(LATENT_SIZE * NUM_INPUT_SLICES, LATENT_SIZE),
        )
    )
data = torch.load("../checkpoints/audio-pred-2/best.pt")
print("steps: {:,}".format(data["num_input_steps"]))
model.load_state_dict(data["state_dict"])
model

# load audio

In [ ]:
filename = "/home/bergi/Music/BR80-backup/ROLAND/LIVEREC/LIVE0033-2023-09-10-poesnek.WAV"
#filename = "/home/bergi/Music/Aphex Twin/Acoustica Alarm Will Sound Performs Aphex Twin/01 Cock_Ver 10.mp3"
filename = "/home/bergi/Music/Ray Kurzweil The Age of Spiritual Machines/(audiobook) Ray Kurzweil - The Age of Spiritual Machines - 1 of 4.mp3"


reader = StreamReader(filename)
reader.add_basic_audio_stream(SLICE_SIZE * NUM_INPUT_SLICES, sample_rate=SAMPLERATE)

reader.seek(1)
play(read_stream(reader.stream(), 10))

# hallucinate

In [ ]:
@torch.no_grad()
def hallucinate(self, audio, num_steps: int = 1):
    assert audio.ndim == 2, audio.ndim
    assert audio.shape[0] == self.channels, audio.shape
    num_audio_slices = audio.shape[1] // self.slice_size

    input_audio = audio[:, -(self.slice_size * self.num_input_slices):]
    input_audio = input_audio.view(self.num_input_slices, self.channels, self.slice_size)
    input_embeddings = self.encoder(input_audio)
    new_audio = []
    for i in range(num_steps):
        embeddings = input_embeddings.view(-1, self.channels, self.latent_dims * self.num_input_slices)
        next_embedding = self.predictor(embeddings)
        
        audio_slice = self.decoder(next_embedding + 0. * next_embedding.std() * torch.randn_like(next_embedding))
        new_audio.append(audio_slice)
        
        next_embedding = next_embedding 
        input_embeddings = torch.concat([input_embeddings[1:], next_embedding[0]])
    
    new_audio = torch.concat(new_audio, dim=-1).squeeze(0)
    return torch.cat([audio, new_audio], dim=-1)

reader.seek(1)
wave = read_stream(reader.stream(), .6)
play(hallucinate(model, wave.unsqueeze(0), 100), normalize=True)
#slices, positions = encode_audio(reader.stream(), seconds=10)
#VF.to_pil_image(signed_to_image(slices.permute(1, 0).unsqueeze(0)))